# Example Notebook

This notebook describes an easy 2d problem, serially estimated.

In [1]:
using SMM
using Distributions
using DataStructures

## Step 1: Initializing a SMMProblem

Let's initialize a SMMProblem. Options are to be passsed via `SMMOptions()`. Here we specify the we want to run the optimizer for 1000 iterations and that we want to save the optimization every 1000 steps (that is, only once; when the estimation is complete).

In [2]:
myProblem = SMMProblem(options = SMMOptions(maxFuncEvals=1000, saveSteps = 1000));

## Options

[TOBECOMPLETED]

## Step 2: Set emprical moments

Empirical moments can be loaded from a .csv file using the function `read_empirical_moments(path::String)`.
The .csv file should should contain at least three columns:
* `name`: empirical moments' names
* `value`: empirical moments' values
* `weights`: weight associated with each parameter value

If, for each moment, `weight` is set to be equal to `value`, the objective funtion to be minimized will be 
the mean percentage deviation of simulated moments from their empirical counterparts. 

In [3]:
dictEmpiricalMoments = read_empirical_moments(joinpath(Pkg.dir("SMM"), "test/empiricalMomentsTest2d.csv"))

DataStructures.OrderedDict{String,Array{Float64,1}} with 2 entries:
  "mean1" => [1.0, 1.0]
  "mean2" => [-1.0, -1.0]

Alternatively, one may construct "manually" empirically directly from Julia:

In [4]:
dictEmpiricalMoments = OrderedDict{String,Array{Float64,1}}()
dictEmpiricalMoments["mean1"] = [1.0; 1.0]
dictEmpiricalMoments["mean2"] = [-1.0; -1.0]

2-element Array{Float64,1}:
 -1.0
 -1.0

2 moments found.


One has to let SMM.jl know that we are going to use the empirical moments stored in `dictEmpiricalMoments`.
Use the function `set_empirical_moments!()` for that purpose:

In [5]:
set_empirical_moments!(myProblem, dictEmpiricalMoments)

DataStructures.OrderedDict{String,Array{Float64,1}} with 2 entries:
  "mean1" => [1.0, 1.0]
  "mean2" => [-1.0, -1.0]

## Step 3. Set priors

Priors for parameter values can be loaded from a .csv file using the function `read_priors(path::String)`. The .csv file should contain at least four columns:
* `name` : name for each parameter
* `value` : initial guess
* `lower_bound` : lower bound for this parameter
* `upper_bound`: upper bound for this parameter

In [6]:
read_priors(joinpath(Pkg.dir("SMM"), "test/priorsTest2d.csv"))

DataStructures.OrderedDict{String,Array{Float64,1}} with 2 entries:
  "mu1" => [0.5, -5.0, 5.0]
  "mu2" => [0.1, -5.0, 5.0]

It is also possible to specify priors manually:

In [7]:
dictPriors = OrderedDict{String,Array{Float64,1}}()
dictPriors["mu1"] = [0., -5.0, 5.0]
dictPriors["mu2"] = [0., -5.0, 5.0]

3-element Array{Float64,1}:
  0.0
 -5.0
  5.0

One has to attach priors values the `SMMProblem` we are going to use.
This is achieved using the function  `set_priors!()`

In [8]:
set_priors!(myProblem, dictPriors)

DataStructures.OrderedDict{String,Array{Float64,1}} with 2 entries:
  "mu1" => [0.0, -5.0, 5.0]
  "mu2" => [0.0, -5.0, 5.0]

## Step 3: Specifying the function that generate simulated moments using parameter values

This function has two constraints:
* it should take as an argument a vector (the vector of parameters)
* it should return an OrderedDict containing the same moments as the ones contained in the OrderedDict 
containing empirical moments (`dictEmpiricalMoments`)

In this example, we simulate draws from a mutlivariate normal with unit variance:

In [9]:
function functionTest2d(x)

    d = MvNormal([x[1]; x[2]], eye(2))
    output = OrderedDict{String,Float64}()

    draws = rand(d, 1000000)
    output["mean1"] = mean(draws[1,:])
    output["mean2"] = mean(draws[2,:])

    return output
end

functionTest2d (generic function with 1 method)

2 prior values found


Let's attach the function `functionTest2d(x)` to our SMMProblem using the function `set_simulate_empirical_moments!()`

In [10]:
set_simulate_empirical_moments!(myProblem, functionTest2d)

functionTest2d (generic function with 1 method)

## Step 4. Constructing the objective function

Having completed steps 1 - 3, it is now time to build the function that the solver will be 
minimizing. This is achieved with the function `construct_objective_function!`

In [11]:
# Construct the objective function using: 
#* the function: parameter -> simulated moments
#* emprical moments values
#* emprical moments weights
    
construct_objective_function!(myProblem)

(::objective_function_percent) (generic function with 1 method)

## Step 5. Running the optimization

It is now time to call the minimization algorithm using the function `smmoptimize!()`.
The optimization is done by `BlackBoxOptim.jl`. By default, the optimizer is `:adaptive_de_rand_1_bin_radiuslimited`.

In [16]:
# Run the optimization
# This function first modifies t.bbSetup
# and then modifies t.bbResults
@time smmoptimize!(myProblem, verbose = true)

1 worker(s) detected
-------------------------------------
Batch 1.0 / 1.0
------------------------------------- 

Starting optimization with optimizer BlackBoxOptim.DiffEvoOpt{BlackBoxOptim.FitPopulation{Float64},BlackBoxOptim.RadiusLimitedSelector,BlackBoxOptim.AdaptiveDiffEvoRandBin{3},BlackBoxOptim.RandomBound{BlackBoxOptim.RangePerDimSearchSpace}}
0.00 secs, 0 evals, 0 steps
DE modify state:
0.56 secs, 16 evals, 8 steps, improv/step: 0.625 (last = 0.6250), fitness=0.508505600
DE modify state:
1.09 secs, 32 evals, 17 steps, improv/step: 0.471 (last = 0.3333), fitness=0.259643169
DE modify state:
1.62 secs, 48 evals, 26 steps, improv/step: 0.500 (last = 0.5556), fitness=0.179299763
DE modify state:
2.14 secs, 64 evals, 36 steps, improv/step: 0.472 (last = 0.4000), fitness=0.138079265
DE modify state:
2.69 secs, 80 evals, 45 steps, improv/step: 0.444 (last = 0.3333), fitness=0.138079265
DE modify state:
3.22 secs, 95 evals, 55 steps, improv/step: 0.418 (last = 0.3000), fitness=0.1380

(Any[2.28221e-5], Any[[0.997929, -0.993436]])

33.943860 seconds (136.84 k allocations: 29.872 GiB, 4.40% gc time)


## Step 6. Analysing Results

### A. Point estimates

The parameter value that minimizes the distance between empirical and simulated moments can be accessed using 
the function `smm_minimizer` 

In [15]:
minimizer = smm_minimizer(myProblem)
println(minimizer[1])
println(minimizer[2])

1.0062686679725348
-0.9950763952944425


### B. Standard errors

[WORKINPROGRESS]

### C. Slices

[WORKINPROGRESS]